## Import data and packages

In [215]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import numpy as np 
import json
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder



# import data
df_base = pd.read_csv('data/Base.csv') 

# import feature data types
with open('python_scripts/data_types.json') as f:
    data_types = json.load(f) 

# encoding source {'INTERNET', 'TELEAPP'} into source_is_internet_not_teleapp
df_base['source_is_internet_not_teleapp'] = (df_base['source'] == 'INTERNET').astype(int)
df_base.drop('source', axis=1, inplace=True)


# recording which variables have missing values recorded as -1
missing_values = ['prev_address_months_count',
                  'current_address_months_count',
                  'bank_months_count',
                  'session_length_in_minutes',
                  'device_distinct_emails_8w']

### Divide dataset into training set and testing set

In [216]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_base.drop('fraud_bool', axis=1), df_base.fraud_bool, test_size=0.2, random_state=0)


### Deal with missing values

In [217]:
# device_distinct_emails_8w - impute missing values with mode (since it is categories, dont necessarily want middle value, just the most represented)
device_distinct_emails_8w_mode = X_train.device_distinct_emails_8w.value_counts().sort_values(ascending=False).index[0]

X_train.loc[X_train.device_distinct_emails_8w == -1, 'device_distinct_emails_8w'] = device_distinct_emails_8w_mode
X_test.loc[X_test.device_distinct_emails_8w == -1, 'device_distinct_emails_8w'] = device_distinct_emails_8w_mode


In [218]:
# session_length_in_minutes - impute missing values with median
session_length_in_minutes_median_position = int(np.ceil((len(X_train[X_train.session_length_in_minutes != -1])/2)-1))
session_length_in_minutes_median = sorted(X_train[X_train.session_length_in_minutes != -1].session_length_in_minutes)[session_length_in_minutes_median_position]

X_train.loc[X_train.session_length_in_minutes == -1, 'session_length_in_minutes'] = session_length_in_minutes_median
X_test.loc[X_test.session_length_in_minutes == -1, 'session_length_in_minutes'] = session_length_in_minutes_median


In [219]:
# drop bank_months_count, create binary feature for value provided or not
X_train['bank_months_count_provided'] = [0 if i == -1 else 1 for i in X_train.bank_months_count]
X_train.drop('bank_months_count', axis=1, inplace=True)

X_test['bank_months_count_provided'] = [0 if i == -1 else 1 for i in X_test.bank_months_count]
X_test.drop('bank_months_count', axis=1, inplace=True)

In [220]:
# drop prev_address_months_count (too many missing values), create binary feature for value provided or not
X_train['prev_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_train.prev_address_months_count]
X_train.drop('prev_address_months_count', axis=1, inplace=True)

X_test['prev_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_test.prev_address_months_count]
X_test.drop('prev_address_months_count', axis=1, inplace=True)

In [221]:
# current_address_months_count - impute missing values with median
X_train['current_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_train.current_address_months_count]
X_test['current_address_months_count_provided'] = [0 if i == -1 else 1 for i in X_test.current_address_months_count]

# and create binary features for value provided or not

current_address_months_count_median_position = int(np.ceil((len(X_train[X_train['current_address_months_count'] != -1])/2)-1))
current_address_months_count_median = sorted(X_train[X_train['current_address_months_count'] != -1]['current_address_months_count'])[current_address_months_count_median_position]

X_train.loc[X_train['current_address_months_count'] == -1, 'current_address_months_count'] = current_address_months_count_median
X_test.loc[X_test['current_address_months_count'] == -1, 'current_address_months_count'] = current_address_months_count_median

### Encode and simplfy categorical features

In [222]:
# drop device_fraud_count (single value)
X_train.drop('device_fraud_count', axis=1, inplace=True)
X_test.drop('device_fraud_count', axis=1, inplace=True)

In [223]:
# simplying proposed credit limit to 5 categories
categories_to_keep = X_train.proposed_credit_limit.value_counts().index[:4] # taking 4 largest represented proposed limits
X_train['proposed_credit_limit'] = X_train['proposed_credit_limit'].where(X_train['proposed_credit_limit'].isin(categories_to_keep), 'other').astype(str)
X_test['proposed_credit_limit'] = X_test['proposed_credit_limit'].where(X_test['proposed_credit_limit'].isin(categories_to_keep), 'other').astype(str)

In [224]:
# one hot encoding
categories_to_encode = ['proposed_credit_limit', 'payment_type', 'employment_status', 'housing_status', 'device_os']
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first') 
encoded_array = ohe.fit_transform(X_train[categories_to_encode])
encoded_df = pd.DataFrame(encoded_array, columns=ohe.get_feature_names_out())
X_train = pd.concat([X_train.drop(columns=categories_to_encode).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

encoded_array_test = ohe.transform(X_test[categories_to_encode])
encoded_df_test = pd.DataFrame(encoded_array_test, columns=ohe.get_feature_names_out())
X_test = pd.concat([X_test.drop(columns=categories_to_encode).reset_index(drop=True), encoded_df_test.reset_index(drop=True)], axis=1)


### Log transformations of skewed features

In [294]:
# test for skewness for possible log transformation
from scipy.stats import skew
numericals = data_types['numerical_continuous_bounded']+data_types['numerical_continuous_unbounded']+data_types['numerical_discrete']+['customer_age']
to_drop = ['prev_address_months_count', 'bank_months_count']
features = []
skews = []
min_values = []
for i in numericals:
    if i not in to_drop:
        features.append(i)
        skews.append(skew(X_train[i]))
        min_values.append(min(X_train[i]))
skews_df = pd.DataFrame({'Feature':features, 'Skewness':skews, 'Min':min_values}).sort_values('Skewness', ascending=False).set_index('Feature')
skews_df


,Skewness,Min
Feature,,
days_since_request,9.278334,4.036860e-09
session_length_in_minutes,3.315531,8.720274e-04
bank_branch_count_8w,2.746926,0.000000e+00
intended_balcon_amount,2.505943,-1.553055e+01
zip_count_4w,1.454840,1.000000e+00
current_address_months_count,1.389416,0.000000e+00
date_of_birth_distinct_emails_4w,0.703265,0.000000e+00
velocity_6h,0.563249,-1.554307e+02
customer_age,0.479187,1.000000e+01


In [296]:
significant_skews = list(skews_df[skews_df.Skewness >= 1].index)
# shift variables to push negative values above 0
shifted_df = {}
for i in significant_skews:
    min_value = skews_df.loc[i].Min 
    if min_value <1 :
        shifted_df[i] = X_train[i] + 1 + abs(min_value)
    else:
        shifted_df[i] = X_train[i]

# check if range of values spans 3 orders of magnitude (largest value is over 10^3 time the smallest value)
features = []
range_orders_of_magnitude = []
for i,j in shifted_df.items():
    log_min = np.log10(min(j))
    log_max = np.log10(max(j))
    range_orders_of_magnitude.append(log_max - log_min)
    features.append(i)
magnitude_of_ranges_df = pd.DataFrame({'Feature': features, 
                                       'range_orders_of_magnitude':range_orders_of_magnitude}).sort_values('range_orders_of_magnitude', ascending=False).set_index('Feature')
magnitude_of_ranges_df = magnitude_of_ranges_df.join(skews_df.drop('Min', axis=1), on='Feature', how='left')
magnitude_of_ranges_df

,range_orders_of_magnitude,Skewness
Feature,,
zip_count_4w,3.826075,1.454840
bank_branch_count_8w,3.377670,2.746926
current_address_months_count,2.629410,1.389416
intended_balcon_amount,2.112228,2.505943
session_length_in_minutes,1.938263,3.315531
days_since_request,1.900132,9.278334


In [310]:
# `current_address_months_count` displays moderate skew and range below 3 orders of magnitude, so I will not log transform this variable
# executing log transformation
to_transform = list(magnitude_of_ranges_df.index.drop(['current_address_months_count']))

# the features will be shifted up by the following values before log transformation occurs (if minimum value of a feature is 1 or above, there will be no shift)
feature_shifts = {}
for i in to_transform:
    min_value = min(X_train[i])
    if min_value < 1:
        feature_shifts[i] = 1 + abs(min_value)
    else:
        feature_shifts[i] = 0

# log transformations 
for i in to_transform:
    # on training set
    X_train[i] = np.log(X_train[i] + feature_shifts[i])

    # on test set using feature shifts defined by training set
    X_test[i] = np.log(X_test[i] + feature_shifts[i])

In [318]:
# compare skewness after transformation
features = []
skews = []
for i in to_transform:
        features.append(i)
        skews.append(skew(X_train[i]))
skews_df = pd.DataFrame({'Feature':features, 'Skewness_After':skews}).set_index('Feature')
magnitude_of_ranges_df.join(skews_df, on='Feature', how='inner').drop('range_orders_of_magnitude', axis=1).rename(columns={'Skewness':'Skewness_Before'})

,Skewness_Before,Skewness_After
Feature,,
zip_count_4w,1.454840,-0.358178
bank_branch_count_8w,2.746926,0.921697
intended_balcon_amount,2.505943,1.567151
session_length_in_minutes,3.315531,0.489976
days_since_request,9.278334,3.827958


### Scaling for numerical features

In [233]:
# apply robust scaling
from sklearn.preprocessing import RobustScaler

# Initialize the RobustScaler
scaler = RobustScaler()

# Scale all numerical features in both train and test sets
# X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
# X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

In [234]:
# ready for training

